In [24]:
from tqdm import trange, notebook
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import WebDriverException

In [25]:
# 라이브러리 불러오기
from selenium import webdriver # 브라우저 자동화를 위한 모듈
import requests # HTTP 요청을 보내기 위한 모듈
from bs4 import BeautifulSoup as BS # HTML 내용 파싱을 위한 모듈
import pandas as pd # 데이터 조작 및 분석을 모듈
import time # 코드 실행 속도 조절을 위한 모듈
import re # 정규 표현식 사용을 위한 모듈
import pickle # 파이썬 객체 직렬화를 위한 모듈
from selenium.webdriver.common.by import By # 다양한 방법으로 엘리먼트를 찾기 위한 모듈
import datetime # 날짜와 시간 연산을 위한 모듈
import time
import numpy as np
import pandas as pd
from threading import Thread
from datetime import datetime
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [26]:
keywords = ['마이크로소프트', '애플', '엔비디아', '구글', '아마존']
keyword = '국민연금'
# start_date = ['20220501','20220601','20220701','20220801','20220901','20221001','20221101','20221201', ]
# end_date =   ['20220531','20220630','20220731','20220831','20220930','20221031','20221130','20221231',]
start_date = ['20220501','20220601','20220701','20220801','20220901','20221001','20221101','20221201', '20230101','20230201','20230301','20230401']
end_date =   ['20220531','20220630','20220731','20220831','20220930','20221031','20221130','20221231', '20230131','20230228','20230331','20230430']

In [27]:
url= []

# for keyword in keywokrds :
for i in range(len(start_date)):
    start, end=start_date[i], end_date[i]
    period = 'from' + start +'to' + end
    url.append('https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query='+ keyword +'&nso=p%3A' + period)

In [28]:
url

['https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=국민연금&nso=p%3Afrom20220501to20220531',
 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=국민연금&nso=p%3Afrom20220601to20220630',
 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=국민연금&nso=p%3Afrom20220701to20220731',
 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=국민연금&nso=p%3Afrom20220801to20220831',
 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=국민연금&nso=p%3Afrom20220901to20220930',
 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=국민연금&nso=p%3Afrom20221001to20221031',
 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=국민연금&nso=p%3Afrom20221101to20221130',
 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=국민연금&nso=p%3Afrom20221201to20221231',
 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query=국민연금&nso=p%3Afrom20230101to202

In [29]:
# 스코롤 다운 함수 정의
def doScrollDown(whileSeconds): 
    start = datetime.datetime.now() # 스크롤 다운 시작 시간 설정
    end = start + datetime.timedelta(seconds=whileSeconds) # 스크롤 다운 종료 시간 설정
    while True:
        # 페이지 맨 아래로 스크롤 다운
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1) # 1초 대기
        
        # 종료 시간에 도달하면 반복 종료
        if datetime.datetime.now() > end:  
            break

In [30]:
# 스크롤 다운 함수에 driver 인자 추가
def endScrollDown(driver):
    end = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        start = driver.execute_script("return document.body.scrollHeight")
        if start == end:
            break
        end = start

In [31]:
# 병렬로 수행될 크롤링 함수
def crawl(url):
    # 크롬 드라이버 실행
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-extensions")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    driver.set_script_timeout(30)  # JavaScript 실행 시간 초과 설정

    global global_new_doc, global_like_cnt, global_comment_cnt, global_comment_list, global_img_cnt, global_div_cnt, global_title_list
    
    driver.get(url)
    endScrollDown(driver)
    titles = driver.find_elements(By.CLASS_NAME, 'title_link')
    
    url_list = []  # URL을 저장할 리스트
    local_new_doc = []
    local_like_cnt = []
    local_comment_cnt = []
    local_comment_list = []
    local_img_cnt = []
    local_div_cnt = []
    local_title_list = []
    
    # 제목과 URL을 저장할 리스트 초기화
    title_list = []  # 제목을 저장할 리스트 초기화
    url_list = []  # URL을 저장할 리스트 초기화

    titles = driver.find_elements(By.CLASS_NAME, 'title_link')  # 현재 페이지에서 class가 'title_link'인 페이지 저장
    for i, title_element in enumerate(titles):
        try:
            title_list.append(title_element.text)  # titles 리스트에서 제목을 추출해 title_list에 추가
            url_list.append(title_element.get_attribute('href'))  # titles 리스트에서 URL을 추출해 url_list에 추가
        except Exception as e:
            print(f"오류 발생: {e}")
            continue  # 오류가 발생해도 다음 항목으로 계속 진행
    
    # 본문, 좋아요 수, 댓글 수, 댓글, 이미지 수, 영상 수를 저장할 리스트를 초기화
    new_doc = []  # 본문을 저장할 리스트 초기화
    like_cnt = []  # 좋아요 수를 저장할 리스트 초기화
    comment_cnt = []  # 댓글 수를 저장할 리스트 초기화
    comment_list = []  # 댓글을 저장할 리스트 초기화
    img_cnt = []  # 이미지 수를 저장할 리스트 초기화
    div_cnt = []  # 영상 수를 저장할 리스트 초기화

    # 카페 글 크롤링
    for i in notebook.tqdm(url_list):
        url_path = i
        driver.switch_to.window(driver.window_handles[0])  # 첫 번째 탭으로 이동
        driver.execute_script("window.open('{}')".format(url_path))  # URL 실행
        driver.switch_to.window(driver.window_handles[1])  # 두 번째 탭으로 이동

        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'iframe')))
            iframes = driver.find_elements(By.TAG_NAME, 'iframe')  # iframe 전환

            if len(iframes) > 0:
                try:
                    # iframe 전환하여 본문과 정보 추출
                    driver.switch_to.frame('cafe_main')
                    html = driver.page_source  # 현재 페이지의 HTML 코드를 가져와서 html 변수에 저장
                    soup = BS(html, 'html.parser')  # 댓글 영역의 HTML 내용을 분석

                    try:
                        # 본문 추출 (article_viewer 클래스를 찾아서 본문 텍스트를 가져옴)
                        a = soup.find('div', class_='article_viewer').get_text()
                    except:
                        a = 'null'

                    try:
                        # 좋아요 수 추출 (u_cnt _count 클래스를 찾아서 좋아요 수 텍스트를 가져옴)
                        b = soup.find('em', class_='u_cnt _count').get_text()
                    except:
                        b = 'null'

                    try:
                        # 댓글 수 추출 (strong num 클래스를 찾아서 댓글 수 텍스트를 가져옴)
                        c = soup.find('strong', class_='num').get_text()
                    except:    
                        c = 'null'

                    try:
                        # 댓글 추출 (span text_comment 클래스를 찾아서 댓글을 가져옴)
                        d = ''
                        commentList = soup.find_all('span', class_='text_comment')
                        for t in commentList:
                            d += t.get_text() + "\n"
                        d += "\n"
                    except:
                        d = 'null'

                    e = len(soup.find_all('img', class_='se-image-resource'))  # 이미지 수 추출
                    f = len(soup.find_all('div', class_='pzp-ui-dimmed pzp-dimmed pzp-pc__dimmed'))  # 영상 수 추출

                    # 추출한 데이터를 각각의 리스트에 추가
                    new_doc.append(a)
                    like_cnt.append(b)
                    comment_cnt.append(c)
                    comment_list.append(d)
                    img_cnt.append(e)
                    div_cnt.append(f)
                    driver.switch_to.default_content()  # iframe에서 기본 컨텐츠로 전환
                except Exception as e:
                    print(f"iframe 처리 중 오류 발생: {e}")
                    new_doc.append('')  # 본문이 없을 경우 빈 문자열 추가
                    like_cnt.append('')  # 좋아요 수가 없을 경우 빈 문자열 추가
                    comment_cnt.append('')  # 댓글 수가 없을 경우 빈 문자열 추가
                    comment_list.append('')  # 댓글이 없을 경우 빈 문자열 추가
                    img_cnt.append('')  # 이미지 수가 없을 경우 빈 문자열 추가
                    div_cnt.append('')  # 영상 수가 없을 경우 빈 문자열 추가
                    driver.switch_to.default_content()
                    continue
            else:
                new_doc.append('null')  # iframe이 없을 경우 'null' 추가

        except UnexpectedAlertPresentException:
            try:
                alert = driver.switch_to.alert
                print("Alert text:", alert.text)
                alert.accept()  # 경고 수락
            except NoAlertPresentException:
                print("No alert present")
            driver.close()  # 탭 닫기
            driver.switch_to.window(driver.window_handles[0])
            continue  # 다음 페이지로 건너뛰기

        except TimeoutException as e:
            print(f"TimeoutException occurred: {e}")
            driver.close()  # 탭 닫기
            driver.switch_to.window(driver.window_handles[0])
            continue  # 다음 페이지로 건너뛰기

        except WebDriverException as e:
            print(f"WebDriverException occurred: {e}")
            driver.close()  # 탭 닫기
            driver.switch_to.window(driver.window_handles[0])
            continue  # 다음 페이지로 건너뛰기

        driver.close()  # 현재 탭 닫기
        driver.switch_to.window(driver.window_handles[0])  # 첫 번째 탭으로 전환

    global_title_list.extend(title_list)
    global_new_doc.extend(new_doc)
    global_like_cnt.extend(like_cnt)
    global_comment_cnt.extend(comment_cnt)
    global_comment_list.extend(comment_list)
    global_img_cnt.extend(img_cnt)
    global_div_cnt.extend(div_cnt)

    driver.quit()  # 크롬 닫기

In [33]:
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
global_title_list=[]
global_new_doc=[]
global_like_cnt=[]
global_comment_cnt=[]
global_comment_list=[]
global_img_cnt=[]
global_div_cnt=[]
#global_new_doc = [] # 본문을 저장할 공유 리스트 초기화
#global_publish_date= [] # 신문 작성 날짜 저장할 공유 리스트 초기화
#global_title_list = []  # 제목을 저장할 공유 리스트
start_time = datetime.now()


threads = []
for i in url:
    thread = Thread(target=crawl, args=(i,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()  # 모든 스레드가 종료될 때까지 대기

end_time= datetime.now()
print("걸린시간: ", end_time - start_time)

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

...

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

Exception in thread Thread-36:
Traceback (most recent call last):
  File "/tmp/ipykernel_22276/1103321578.py", line 57, in crawl
  File "/opt/conda/envs/py39/lib/python3.9/site-packages/selenium/webdriver/support/wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
#0 0x55d2264bef33 <unknown>
#1 0x55d2261b6ce6 <unknown>
#2 0x55d226201e48 <unknown>
#3 0x55d226201f01 <unknown>
#4 0x55d2262453f4 <unknown>
#5 0x55d226223edd <unknown>
#6 0x55d226242899 <unknown>
#7 0x55d226223c53 <unknown>
#8 0x55d2261f4db3 <unknown>
#9 0x55d2261f577e <unknown>
#10 0x55d22648486b <unknown>
#11 0x55d226488885 <unknown>
#12 0x55d226472181 <unknown>
#13 0x55d226489412 <unknown>
#14 0x55d22645625f <unknown>
#15 0x55d2264ad528 <unknown>
#16 0x55d2264ad723 <unknown>
#17 0x55d2264be0e4 <unknown>
#18 0x7fa64dafab43 <unknown>


During handling of the above exception, another exception occurred:

Traceback (most recent 

In [ ]:
# 데이터프레임 생성을 위한 각 리스트의 길이 확인
lengths = [len(global_title_list), len(global_new_doc), len(global_like_cnt), 
           len(global_comment_cnt), len(global_comment_list), len(global_img_cnt), 
           len(global_div_cnt)]
lengths

In [ ]:
min_length = min(lengths)

# 최소 길이에 맞게 각 리스트 자르기
global_title_list = global_title_list[:min_length]
global_new_doc = global_new_doc[:min_length]
global_like_cnt = global_like_cnt[:min_length]
global_comment_cnt = global_comment_cnt[:min_length]
global_comment_list = global_comment_list[:min_length]
global_img_cnt = global_img_cnt[:min_length]
global_div_cnt = global_div_cnt[:min_length]

# 데이터프레임 생성
raw_data = pd.DataFrame({
    'title': global_title_list,
    'doc': global_new_doc,
    'like': global_like_cnt,
    'comment_cnt': global_comment_cnt,
    'comment_list': global_comment_list,
    'img': global_img_cnt,
    'div': global_div_cnt,
    'ch': 'naver',
    'ch2': 'cafe'
})

In [ ]:
# 크롤링 데이터를 데이터프레임으로 변환
# raw_data = pd.DataFrame()
# raw_data['title'] = global_title_list
# raw_data['doc'] = global_new_doc
# raw_data['like'] = global_like_cnt
# raw_data['comment_cnt'] = global_comment_cnt
# raw_data['comment_list'] = global_comment_list
# raw_data['img'] = global_img_cnt
# raw_data['div'] = global_div_cnt
# raw_data['ch'] = 'naver'
# raw_data['ch2'] = 'cafe'

In [ ]:
pd.isna(raw_data['title']).sum()

In [ ]:
pd.isna(raw_data['doc']).sum()

In [ ]:
raw_data.loc[pd.isna(raw_data['doc']),:]

In [ ]:
raw_data.loc[pd.isna(raw_data['title']),:]

In [ ]:
# 데이터프레임을 pickle 파일로 저장
f = open("../data/raw_data/pension_cafe.pkl", "wb")
pickle.dump(raw_data,f)
f.close()